# Lab: Choropleth Maps {#sec-choropleth-maps}

A visualisation often shown is a choropleth. This is a series of spatial polygons (such as states in the USA) which are coloured by a feature, like the one below. 

![A choropleth map showing CO2 emissions by country as compared to world average. Source [Our World in Data](https://ourworldindata.org/grapher/per-capita-co2-vs-average)](figs/per-capita-co2-vs-average.png)

In this lab, we will look at creating choropleths of polling data in the recent USA election, and how maps can sometimes be deceptive (as well as how to detect -and avoid- such techniques). To do so, we will be using `geopandas` for the geospatial features, and `altair` for the maps' visualisations.

::: callout-note
### About geopandas

[`geopandas`](https://geopandas.org) is a very specific and complex library that is not installed by default in Anaconda, so normally you would need to install it (and its multiple dependencies) by yourselves. If you are using the course's virtual environment, this should be installed for you the first time you set up your environment for the module. Refer to @sec-setup for instructions on how to set up your environment.

:::

## Data preparations

We will be loading two datasets:

1. `geo_states`: contains the geospatial polygons of the states in America, but does not contain any data about USA elections;
2. `df_polls`: the polling data we used in the last notebook, but does not have any geospatial polygons (you can find more information about every variable [here](https://github.com/fivethirtyeight/data/tree/master/election-forecasts-2020)). 

In [ ]:
import geopandas as gpd 
import pandas as pd
import altair as alt

geo_states = gpd.read_file('data/gz_2010_us_040_00_500k.json')
df_polls = pd.read_csv('data/presidential_poll_averages_2020.csv')

Let's explore the data first:

In [ ]:
geo_states.head()

This seems like a regular data frame, but there's a feature that stands out from the others: `geometry`. This feature contains the coordinates thar define the polygons (or multipolygons) for every region in the map, in this case, every State in the USA. This is also an indicator that we are not using a regular dataframe, but a particular type of dataframe called `GeoDataFrame`:

In [ ]:
type(geo_states)

Because this is a geospatial dataframe, we can visualise it as a map. In this case, we are going to use Altair to create a map using the AlbersUsa projection.

In [ ]:
alt.Chart(geo_states, title='US states').mark_geoshape().encode(
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

And now the polls' result:

In [ ]:
df_polls

As you can see, `modeldate` has different dates. Let's double check that:

In [ ]:
df_polls.modeldate.unique()

### Filtering

That means, that we will need to filter our poll data to a specific date, in this case `11/2/2020`

In [ ]:
df_nov = df_polls[
    (df_polls.modeldate == '11/3/2020')
]

df_nov_states = df_nov[
    (df_nov.candidate_name == 'Donald Trump') |
    (df_nov.candidate_name == 'Joseph R. Biden Jr.')
]

df_nov_states

### Computing percentages

We want to put the percentage estimates for each candidate onto the map. First, let us create a dataframe containing the data for each candidate.

In [ ]:
# Create separate data frame for Trump and Biden
trump_data = df_nov_states[
    df_nov_states.candidate_name == 'Donald Trump'
]

biden_data = df_nov_states[
    df_nov_states.candidate_name == 'Joseph R. Biden Jr.'
]

### Joining data

As we have seen before, we have two datasets that partially address our needs:  `geo_states` contains the geospatial polygons of the states in America, but lacks data about USA elections; `df_polls` contains data about USA elections but lacks geometry. 

We will need to combine both (joining) to create a (geospatial)dataframe that contains geometry AND polling data so we can create a choropleth map capable of answering our question: _who is winning the elections?_

To do so, we need to join both dataframes using a common feature. Our spatial and poll data have the name of the state in common, but their columns have different names. 

We could rename the columns names, and then join them with `pd.merge()` but instead, we are going to use a less destructive way.

We can join the geospatial data and poll data using `pd.merge()` while providing different column names by using `left_on` for the left data (usually the geodataframe) and `right_on` for the right dataframe. We will be using this method, as it doesn't require to rename columns.

In [ ]:
# Add the poll data (divided in two data frames) to a single geospatial dataframe.
geo_states_trump = geo_states.merge(
    trump_data, left_on = 'NAME', right_on = 'state')

geo_states_biden = geo_states.merge(
    biden_data, left_on = 'NAME', right_on = 'state')

In [ ]:
geo_states_trump.head()

In [ ]:
geo_states_biden.head()

Joe Biden is clearly winning. Can we make it look like he is not?

## Data visualisation

We can plot this specifying the feature to use for our colour.

In [ ]:
#| label: fig-default-choropleth-map
#| fig-catption: Default Choropleth map using Altair.

alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    color='pct_estimate',
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

### Binning

To smooth out any differences we can bin our data.

In the case below, we will be binning based on a single value (step):

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate', bin=alt.Bin(step=35)),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

:::callout-caution

## Your turn

How would you interpret the plot above? 
What would change if we change the value of the step?

:::

What about if we increase the binstep so we have more bins?

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate', bin=alt.Bin(step=5)),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

::: callout-caution

## Your turn

Try different step sizes for the bins and consider how bins can shape our interpretation of the data. What would happen if plots with different bin sizes were placed side to side?

:::


To add further confusion, what happens when we log scale the data?

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate', bin=alt.Bin(step=5), scale=alt.Scale(type='log')),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

vs

In [ ]:
alt.Chart(geo_states_biden, title='Poll estimate for Joe Biden on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate', bin=alt.Bin(step=5), scale=alt.Scale(type='log')),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

What is happening here?!?!

### Colour palettes

Next up, what about the colours we use and the range of values assigned to each color? Code inspired by/taken from [here](https://colab.research.google.com/drive/1PePamFUfrgvN3ZYaN8fWfP8ovIJ0gyre#scrollTo=Poo1da-8u3cX).

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donal Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate',
    scale=alt.Scale(type="linear",
              domain=[10, 40, 50, 55, 60, 61, 62],
                          range=["#414487","#414487",
                                 "#355f8d","#355f8d",
                                 "#2a788e",
                                 "#fde725","#fde725"])),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

Compare that with

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate',
    scale=alt.Scale(type="linear",
              domain=[10, 20, 30, 35, 68, 70, 100],
                          range=["#414487","#414487",
                                 "#7ad151","#7ad151",
                                 "#bddf26",
                                 "#fde725","#fde725"])),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

### Legends

My goodness! So what have we played around with?

* Transforming our scale using log
* Binning our data to smooth out variances
* Altering our colour scheme and the ranges for each colour

... what about if we remove the legend?

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate',
    scale=alt.Scale(type="linear",
              domain=[10, 20, 30, 35, 68, 70, 100],
                          range=["#414487","#414487",
                                 "#7ad151","#7ad151",
                                 "#bddf26",
                                 "#fde725","#fde725"]),
                                 legend=None),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

Good luck trying to interpret that. Though we often see maps without legends and with questionable colour schemes on TV.

::: callout-caution

## Food for thought

How do you think choropleths should be displayed? What information does a use need to understand the message communicated in these plots?

:::